# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

M. Guedel  ->  M. Guedel  |  ['M. Guedel']
J. Lian  ->  J. Lian  |  ['J. Lian']
J. Neumann  ->  J. Neumann  |  ['J. Neumann']
N. Winkel  ->  N. Winkel  |  ['N. Winkel']


A. Gould  ->  A. Gould  |  ['A. Gould']
Arxiv has 68 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2307.13722


extracting tarball to tmp_2307.13722...

 done.
Retrieving document from  https://arxiv.org/e-print/2307.13887


extracting tarball to tmp_2307.13887...

...

Found 87 bibliographic references in tmp_2307.14139/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2307.14274


extracting tarball to tmp_2307.14274...

 done.


/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:120: LatexWarning: attempting recovering figure 4_Light_Curve_Modeling/lc2.eps
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:692: LatexWarning: Could not find graphic \includegraphics[scale=0.6,angle=-90]{4_Light_Curve_Modeling/lc2.eps}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))


Found 112 bibliographic references in tmp_2307.14274/ms.bbl.
syntax error in line 214: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.13887-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.13887) | **A Tale of Two Disks: Mapping the Milky Way with the Final Data Release  of APOGEE**  |
|| J. Imig, et al. -- incl., <mark>J. Lian</mark> |
|*Appeared on*| *2023-07-27*|
|*Comments*| *41 pages, 32 figures, accepted to ApJ*|
|**Abstract**| We present new maps of the Milky Way disk showing the distribution of metallicity ([Fe/H]), $\alpha$-element abundances ([Mg/Fe]), and stellar age, using a sample of 66,496 red giant stars from the final data release (DR17) of the Apache Point Observatory Galactic Evolution Experiment (APOGEE) survey. We measure radial and vertical gradients, quantify the distribution functions for age and metallicity, and explore chemical clock relations across the Milky Way for the low-$\alpha$ disk, high-$\alpha$ disk, and total population independently. The low-$\alpha$ disk exhibits a negative radial metallicity gradient of $-0.06 \pm 0.001$ dex kpc$^{-1}$, which flattens with distance from the midplane. The high-$\alpha$ disk shows a flat radial gradient in metallicity and age across nearly all locations of the disk. The age and metallicity distribution functions shift from negatively skewed in the inner Galaxy to positively skewed at large radius. Significant bimodality in the [Mg/Fe]-[Fe/H] plane and in the [Mg/Fe]-age relation persist across the entire disk. The age estimates have typical uncertainties of $\sim0.15$ in $\log$(age) and may be subject to additional systematic errors, which impose limitations on conclusions drawn from this sample. Nevertheless, these results act as critical constraints on galactic evolution models, constraining which physical processes played a dominant role in the formation of the Milky Way disk. We discuss how radial migration predicts many of the observed trends near the solar neighborhood and in the outer disk, but an additional more dramatic evolution history, such as the multi-infall model or a merger event, is needed to explain the chemical and age bimodality elsewhere in the Galaxy. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.14139-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.14139) | **Still alive and kicking: A significant outburst in changing-look AGN Mrk  1018**  |
|| R. Brogan, et al. -- incl., <mark>J. Neumann</mark>, <mark>N. Winkel</mark> |
|*Appeared on*| *2023-07-27*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**| Changing-look active galactic nuclei (CL-AGN) have been observed to change optical spectral type. Mrk 1018 is unique: first classified as a type 1.9 Seyfert galaxy, it transitioned to a type 1 before returning to its initial classification after approximately 30 years. We present a high-cadence monitoring programme that caught a major outburst in 2020. Due to sunblock, only the decline could be observed. We studied X-ray, UV, optical, and IR before and after the outburst to investigate the responses of the AGN structures. We derived a u'-band light curve of the AGN contribution alone. The flux increased by a factor of the order of 13. We confirmed this in other optical bands and determined the shape and speed of the decline in each waveband. The shapes of H beta and H alpha were analysed before and after the event. Two XMM-Newton observations from before and after the outburst were also exploited. The outburst is asymmetric, with a swifter rise than decline. The decline is best fit by a linear function, ruling out a tidal disruption event. The optical spectrum shows no change approximately 8 months before and 17 months after. The UV flux increased slightly after the outburst but the X-ray primary flux is unchanged. However, the 6.4 keV Iron line has doubled in strength. IR data taken 13 days after the observed optical peak show an increased emission level. Calculating the distance of the broad-line region and inner edge of the torus from the supermassive black hole can explain the multi-wavelength response to the outburst, in particular: i) the unchanged H beta and H alpha lines, ii) the unchanged primary X-ray spectral components, iii) the rapid and extended infrared response, as well as iv) the enhanced emission of the reflected 6.4 keV line. The outburst was due to a dramatic and short-lasting change in the intrinsic accretion rate. We discuss different models as potential causes. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.14274-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.14274) | **OGLE-2019-BLG-0825: Constraints on the Source System and Effect on  Binary-lens Parameters arising from a Five Day Xallarap Effect in a Candidate  Planetary Microlensing Event**  |
|| Y. K. Satoh, et al. -- incl., <mark>A. Gould</mark> |
|*Appeared on*| *2023-07-27*|
|*Comments*| *19 pages, 7 figures, 6 tables. Accepted by AJ*|
|**Abstract**| We present an analysis of microlensing event OGLE-2019-BLG-0825. This event was identified as a planetary candidate by preliminary modeling. We find that significant residuals from the best-fit static binary-lens model exist and a xallarap effect can fit the residuals very well and significantly improves $\chi^2$ values. On the other hand, by including the xallarap effect in our models, we find that binary-lens parameters like mass-ratio, $q$, and separation, $s$, cannot be constrained well. However, we also find that the parameters for the source system like the orbital period and semi major axis are consistent between all the models we analyzed. We therefore constrain the properties of the source system better than the properties of the lens system. The source system comprises a G-type main-sequence star orbited by a brown dwarf with a period of $P\sim5$ days. This analysis is the first to demonstrate that the xallarap effect does affect binary-lens parameters in planetary events. It would not be common for the presence or absence of the xallarap effect to affect lens parameters in events with long orbital periods of the source system or events with transits to caustics, but in other cases, such as this event, the xallarap effect can affect binary-lens parameters. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.13722-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.13722) | **Formation of a wide-orbit giant planet in a gravitationally unstable  subsolar-metallicity protoplanetary disc**  |
|| R. Matsukoba, E. I. Vorobyov, T. Hosokawa, <mark>M. Guedel</mark> |
|*Appeared on*| *2023-07-27*|
|*Comments*| *11 pages, 8 figures, submitted to MNRAS*|
|**Abstract**| Direct imaging observations of planets revealed that wide-orbit ($>10$ au) giant planets exist even around subsolar-metallicity host stars and do not require metal-rich environments for their formation. A possible formation mechanism of wide-orbit giant planets in subsolar-metallicity environments is the gravitational fragmentation of massive protoplanetary discs. Here, we follow the long-term evolution of the disc for 1 Myr after its formation, which is comparable to disc lifetime, by way of a two-dimensional thin-disc hydrodynamic simulation with the metallicity of 0.1 ${\rm Z}_{\odot}$. We find a giant protoplanet that survives until the end of the simulation. The protoplanet is formed by the merger of two gaseous clumps at $\sim$0.5 Myr after disc formation, and then it orbits $\sim$200 au from the host star for $\sim$0.5 Myr. The protoplanet's mass is $\sim$10 ${\rm M}_{\rm J}$ at birth and gradually decreases to 1 ${\rm M}_{\rm J}$ due to the tidal effect from the host star. The result provides the minimum mass of 1 ${\rm M}_{\rm J}$ for protoplanets formed by gravitational instability in a subsolar-metallicity disc. We anticipate that the mass of a protoplanet experiencing reduced mass loss thanks to the protoplanetary contraction in higher resolution simulations can increase to $\sim$10 ${\rm M}_{\rm J}$. We argue that the disc gravitational fragmentation would be a promising pathway to form wide-orbit giant planets with masses of $\ge1$ ${\rm M}_{\rm J}$ in subsolar-metallicity environments. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2307.13887.md
    + _build/html/tmp_2307.13887/./figures/age_metalicity_relation.png
    + _build/html/tmp_2307.13887/./figures/apogee_mhplots.png
    + _build/html/tmp_2307.13887/./figures/apogee_MDF_annotated_alphas_flip2.png
exported in  _build/html/2307.14139.md
    + _build/html/tmp_2307.14139/./stella_lc.png
    + _build/html/tmp_2307.14139/./wise_atlas.png
    + _build/html/tmp_2307.14139/./host+agn.png
exported in  _build/html/2307.14274.md
    + _build/html/tmp_2307.14274/./lightcurve.png
    + _build/html/tmp_2307.14274/./cmd_e=0.png
    + _build/html/tmp_2307.14274/./GS1+2.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\alpham}{[Mg/Fe]}$</div>



<div id="title">

# A Tale of Two Disks: Mapping the Milky Way with the Final Data Release of APOGEE

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2307.13887-b31b1b.svg)](https://arxiv.org/abs/2307.13887)<mark>Appeared on: 2023-07-27</mark> -  _41 pages, 32 figures, accepted to ApJ_

</div>
<div id="authors">

J. Imig, et al. -- incl., <mark>J. Lian</mark>

</div>
<div id="abstract">

**Abstract:** We present new maps of the Milky Way disk showing the distribution of metallicity ( ${\feh}$ ), $\alpha$ -element abundances ( ${\alpham}$ ), and stellar age, using a sample of \textcolor{black}{66,496} red giant stars from the final data release (DR17) of the Apache Point Observatory Galactic Evolution Experiment (APOGEE) survey. We measure radial and vertical gradients, quantify the distribution functions for age and metallicity, and explore chemical clock relations across the Milky Way for the low- $\alpha$ disk, high- $\alpha$ disk, and total population independently. The low- $\alpha$ disk exhibits a negative radial metallicity gradient of \textcolor{black}{$-0.06 \pm 0.001$} dex kpc $^{-1}$ , which flattens with distance from the midplane. The high- $\alpha$ disk shows a flat radial gradient in metallicity and age across nearly all locations of the disk. The age and metallicity distribution functions shift from negatively skewed in the inner Galaxy to positively skewed at large radius. Significant bimodality in the ${\alpham}$ - ${\feh}$ plane and in the ${\alpham}$ -age relation persist across the entire disk. The age estimates have typical uncertainties of $\sim0.15$ in $\log$ (age) and may be subject to additional systematic errors, which impose limitations on conclusions drawn from this sample. Nevertheless, these results act as critical constraints on galactic evolution models, constraining which physical processes played a dominant role in the formation of the Milky Way disk. We discuss how radial migration predicts many of the observed trends near the solar neighborhood and in the outer disk, but an additional more dramatic evolution history, such as the multi-infall model or a merger event, is needed to explain the chemical and age bimodality elsewhere in the Galaxy.

</div>

<div id="div_fig1">

<img src="tmp_2307.13887/./figures/age_metalicity_relation.png" alt="Fig25" width="100%"/>

**Figure 25. -** The age-metallicity relation across the Milky Way disk. Panels represent different spatial zones, laid out in the same way as Figure \ref{fig:apogee_mhplots}, with rows corresponding to $Z$ and columns increasing in $R$. The number in the top-right corner of each panel is the number of stars in our sample in that spatial bin. The age and metallicity for individual stars is plotted, colored by {$\alpham$} abundance. The running median trend is plotted in black square points to guide the eye, with the vertical bars indicating the standard deviation in {\feh} for bins in log(age). The typical (median) uncertainty for any given point is shown in the top right corner of each panel. (*fig:age_metallicity_relation*)

</div>
<div id="div_fig2">

<img src="tmp_2307.13887/./figures/apogee_mhplots.png" alt="Fig21" width="100%"/>

**Figure 21. -** The distribution of stars in the {$\alpham$} vs. {\feh} plane as a function of $R$ and $|Z|$, as a contour map of point density. Spatial bins move from closest to the Galactic plane (bottom row, 0.0 $<|Z|<$ 0.5 kpc) to farthest above the Galactic plane (top row, 1.0 $<|Z|<$ 2.0 kpc), and from close to the Galactic center (left column, 0.0 $<|R|<$ 3.0 kpc) to farthest out in the disk (right column, 15.0 $<|R|<$ 25.0 kpc). The number in the top-right corner of each panel is the number of stars in our sample in that spatial bin. For reference, the gray background shape and black line is the same in each panel, to highlight how the sequence changes across location in the Galaxy. The black line is the boundary between high- and low-$\alpha$ populations defined in Equation \ref{eq:alpha_split}, and the gray shape is the contour containing 90\% of the points in the full sample. The typical uncertainties in abundance measurements as a function of metallicity are shown as a $\pm1\sigma$ value at the bottom of each panel for reference. (*fig:apogee_mhplots*)

</div>
<div id="div_fig3">

<img src="tmp_2307.13887/./figures/apogee_MDF_annotated_alphas_flip2.png" alt="Fig14" width="100%"/>

**Figure 14. -** The metallicity distribution function of the Milky Way disk, split into different height bins (panels), from closest to the Galactic plane (top) to farthest beyond (bottom). Each panel shows the fraction of stars at each metallicity {\feh} as a function of Galactocentric radius, further split by color between high-$\alpha$(red) and low-$\alpha$(blue) samples. Every third row is annotated with markings for the peak (or mode) of the distribution (white diamond), as well as the 25th, 50th (median), and 75th percentiles (white tick marks) to highlight the shape of the distribution. (*fig:apogee_MDF_alpha*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2307.13887"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\arraystretch}{2}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{2}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$</div>



<div id="title">

# Still alive and kicking: A significant outburst in changing-look AGN Mrk 1018

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2307.14139-b31b1b.svg)](https://arxiv.org/abs/2307.14139)<mark>Appeared on: 2023-07-27</mark> -  _Accepted for publication in A&A_

</div>
<div id="authors">

R. Brogan, et al. -- incl., <mark>J. Neumann</mark>, <mark>N. Winkel</mark>

</div>
<div id="abstract">

**Abstract:** Changing-look active galactic nuclei (AGN) have been observed to change their optical spectral type. Mrk 1018 is particularly unique: first classified as a type 1.9 Seyfert galaxy, it transitioned to being a type 1 Seyfert galaxy a few years later before returning to its initial classification as a type 1.9 Seyfert galaxy after $\sim$ 30 years. We present the results of a high-cadence optical monitoring programme that caught a major outburst in 2020. Due to sunblock, only the decline could be observed for $\sim$ 200 days. We studied X-ray, UV, optical, and infrared data before and after the outburst to investigate the responses of the AGN structures. We derived a $u'$ -band light curve of the AGN contribution alone. The flux increased by a factor of $\sim$ 13. We confirmed this in other optical bands and determined the shape and speed of the decline in each waveband. The shapes of H $\beta$ and H $\alpha$ were analysed before and after the event. Two _XMM-Newton_ observations (X-ray and UV) from before and after the outburst were also exploited. The outburst is asymmetric, with a swifter rise than decline. The decline is best fit by a linear function, ruling out a tidal disruption event. The optical spectrum shows no change approximately eight months before and 17 months after. The UV flux is increased slightly after the outburst but the X-ray primary flux is unchanged. However, the 6.4 keV Iron line has doubled in strength. Infrared data taken 13 days after the observed optical peak already show an increased emission level as well. Calculating the distance of the broad-line region and inner edge of the torus from the supermassive black hole can explain the multi-wavelength response to the outburst, in particular: i) the unchanged H $\beta$ and H $\alpha$ lines, ii) the unchanged primary X-ray spectral components, iii) the rapid and extended infrared response, as well as iv) the enhanced emission of the reflected 6.4 keV line. The outburst was due to a dramatic and short-lasting change in the intrinsic accretion rate. We discuss different models as potential causes.

</div>

<div id="div_fig1">

<img src="tmp_2307.14139/./stella_lc.png" alt="Fig4" width="100%"/>

**Figure 4. -** Final host-subtracted $u'$-band light curve of the AGN in Mrk 1018. The blue points are the data points with statistical uncertainties shown by error bars and the blue shaded area represents the systematic uncertainty arising from the choice of image used to model the host galaxy component. This outburst is the most significant as yet observed during Mrk 1018's new type 1.9 state. Both sunblock periods are indicated in grey. (*stella_lc*)

</div>
<div id="div_fig2">

<img src="tmp_2307.14139/./wise_atlas.png" alt="Fig9" width="100%"/>

**Figure 9. -** _WISE_ IR light curve from MJD 57957 to MJD 59417 (Jul 2017 -- Jul 2021). The ATLAS $o$-band data for the same time period are shown for a visual reference of the rapidness of the IR response. The green line indicates the observed peak of the optical outburst in the ATLAS $o$-band. The blue dashed lines indicate the dates of the two _XMM-Newton_ observations before and after the outburst. (*wiselc*)

</div>
<div id="div_fig3">

<img src="tmp_2307.14139/./host+agn.png" alt="Fig10" width="100%"/>

**Figure 10. -** Image created by stacking the 20 VIMOS exposures of Mrk 1018 during the faint (type 1.9) phase, shown with a log scale. The pixel scale is $0.205$\arcsec$$, corresponding to 179 \si{pc} per pixel. The red boxes indicate regions that are masked in the fitting process. A subtle light-absorbing structure is marked with a blue box. This is approximated in the fitting by an elongated Sérsic function with a negative intensity.  (*vimos*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2307.14139"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# OGLE-2019-BLG-0825: Constraints on the Source System and Effect on Binary-lens Parameters arising from a Five Day Xallarap Effect in a Candidate Planetary Microlensing Event

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2307.14274-b31b1b.svg)](https://arxiv.org/abs/2307.14274)<mark>Appeared on: 2023-07-27</mark> -  _19 pages, 7 figures, 6 tables. Accepted by AJ_

</div>
<div id="authors">

Y. K. Satoh, et al. -- incl., <mark>A. Gould</mark>

</div>
<div id="abstract">

**Abstract:** We present an analysis of microlensing event OGLE-2019-BLG-0825.This event was identified as a planetary candidate by preliminary modeling.We find that significant residuals from the best-fit static binary-lens model exist and a xallarap effect can fit the residuals very well and significantly improves $\chi^2$ values.On the other hand, by including the xallarap effect in our models, we find that binary-lens parameters like mass-ratio, $q$ , and separation, $s$ , cannot be constrained well.However, we also find that the parameters for the source system like the orbital period and semi major axis are consistent between all the models we analyzed.We therefore constrain the properties of the source system better than the properties of the lens system.The source system comprises a G-type main-sequence star orbited by a brown dwarf with a period of $P\sim5$ days.This analysis is the first to demonstrate that the xallarap effect does affect binary-lens parameters in planetary events.It would not be common for the presence or absence of the xallarap effect to affect lens parameters in events with long orbital periods of the source system or events with transits to caustics, but in other cases, such as this event, the xallarap effect can affect binary-lens parameters.

</div>

<div id="div_fig1">

<img src="tmp_2307.14274/./lightcurve.png" alt="Fig3" width="100%"/>

**Figure 3. -** (Top panel) Light curve for OGLE-2019-BLG-0825.
    Error bars are renormalized according to Equation (\ref{eq:error}).
    The red solid, blue dashed, orange solid, and green dashed lines are the best 2L1S + xallarap model, the best 1L1S + xallarap model, the best 2L1S + parallax model and the best standard 2L1S model described in Section \ref{sec:light_curve_modeling}, respectively.
    (Middle panel) Residuals from the best 2L1S + xallarap model.
    (Bottom panel) Residuals from the best 2L1S + xallarap model binned by 0.2 days.
     (*fig:lightcurve*)

</div>
<div id="div_fig2">

<img src="tmp_2307.14274/./cmd_e=0.png" alt="Fig2" width="100%"/>

**Figure 2. -** 
    Color Magnitude Diagram (CMD, black dots) of the OGLE-$\mathrm{I}\hspace{-1.2pt}\mathrm{I}\hspace{-1.2pt}\mathrm{I}$ stars within $2’$ around OGLE-2019-BLG-0825. The green dots are stars in Baade's window based on Hubble Space Telescope observations \citep{Holtzman+1998}, color- and magnitude-matched at the RCG position. The orange circles represent the positions of the source, and the red dots represent the positions of the RCG centroid within $2’$ around OGLE-2019-BLG-0825.
     (*fig:cmd*)

</div>
<div id="div_fig3">

<img src="tmp_2307.14274/./GS1+2.png" alt="Fig4" width="100%"/>

**Figure 4. -** 
    Map of $\Delta\chi^2$ in each $s–q$ grid from the $(q,s,\alpha)$ grid search for the standard 2L1S model (Left) and for the 2L1S + xallarap model (Right).
    The best fit $\alpha$ is chosen for each grid location, respectively.
    In the map of the standard 2L1S model, we found the best solution at $q\sim10^{-3}$.
    However, for the 2L1S + xallarap map, best solutions at two other local minima appear at $q > 0.1$.
     (*fig:Grid_search*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2307.14274"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

237  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6

  6 publications selected.
